In [1]:
nombre = "\nMadrigal Urencio Ricardo\n"

In [5]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

dir_hdfs = "hdfs://namenode:9000/tmp/amd/locatellimpio"

df = spark.read.csv(dir_hdfs, header = True, inferSchema = True)

print(nombre)

df.createOrReplaceTempView("locatel")
 
schema = StructType([
    StructField("d_codigo", StringType(), True),
    StructField("d_asenta", StringType(), True),
    StructField("d_tipo_asenta", StringType(), True),
    StructField("D_mnpio", StringType(), True),
    StructField("d_estado", StringType(), True),
    StructField("d_ciudad", StringType(), True),
    StructField("d_CP", StringType(), True),
    StructField("c_estado", StringType(), True),
    StructField("c_oficina", StringType(), True),
    StructField("c_CP", StringType(), True),
    StructField("c_tipo_asenta", StringType(), True),
    StructField("c_mnpio", StringType(), True),
    StructField("id_asenta_cpcons", StringType(), True),
    StructField("d_zona", StringType(), True),
    StructField("c_cve_ciudad", StringType(), True)
])

sepomex_df = spark.read\
            .option("header","true")\
            .option("delimiter", "|")\
            .option("skipRows", "1")\
            .option("encoding", "UTF-8")\
            .schema(schema)\
            .csv("hdfs://namenode:9000/tmp/amd/sepomex/")

sepomex_df.count()
sepomex_df.select(['d_codigo', 'D_mnpio', 'd_estado', 'd_ciudad']).distinct().createOrReplaceTempView("sepomex")
spark.sql("show tables").show()


Madrigal Urencio Ricardo

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|         |  locatel|       true|
|         |  sepomex|       true|
+---------+---------+-----------+



In [9]:
sqlstr = """
select coalesce(s.d_ciudad, 'TOTAL') as ciudad,
coalesce(l.tema_solicitud, 'TOTAL') as tema_solicitud,
count(*) as nr,
grouping_id(s.d_ciudad, l.tema_solicitud) as grupo_id
from locatel l
join sepomex s on(l.d_codigo = s.d_codigo)
group by cube(s.d_ciudad, l.tema_solicitud)
order by 1, 2
"""

spark.sql(sqlstr).show(40, truncate = False)

+------------------+---------------------------------------+---+--------+
|ciudad            |tema_solicitud                         |nr |grupo_id|
+------------------+---------------------------------------+---+--------+
|Acapulco de Juárez|ASISTENCIA SOCIAL                      |2  |0       |
|Acapulco de Juárez|COVID                                  |2  |0       |
|Acapulco de Juárez|INFORMACIÓN SOBRE TRÁMITES             |2  |0       |
|Acapulco de Juárez|QUEJAS Y DENUNCIAS                     |4  |0       |
|Acapulco de Juárez|REGISTRO CIVIL                         |1  |0       |
|Acapulco de Juárez|TIERRA, PREDIOS Y USO DE SUELO         |1  |0       |
|Acapulco de Juárez|TOTAL                                  |21 |1       |
|Acapulco de Juárez|TRAMITES / INFO VEHICULAR              |6  |0       |
|Acapulco de Juárez|VEHÍCULOS AUTOMOTORES Y TAXIS          |3  |0       |
|Acatlán de Juárez |SOLICITUD/SEGURO DE DESEMPLEO          |1  |0       |
|Acatlán de Juárez |TOTAL             

In [12]:
sqlstr = """
select coalesce(s.d_ciudad, 'TOTAL_CD') as ciudad,
coalesce(l.tema_solicitud, 'TOTAL_TS') as tema_solicitud,
coalesce(l.sexo, 'TOTAL_SX') as sexo,
count(*) as nr,
grouping_id(s.d_ciudad, l.tema_solicitud, l.sexo) as grupo_id
from locatel l
join sepomex s on(l.d_codigo = s.d_codigo)
group by cube(s.d_ciudad, l.tema_solicitud, l.sexo)
order by 1, 3, 2
"""

spark.sql(sqlstr).show(80, truncate = False)

+------------------+---------------------------------------+---------------+---+--------+
|ciudad            |tema_solicitud                         |sexo           |nr |grupo_id|
+------------------+---------------------------------------+---------------+---+--------+
|Acapulco de Juárez|ASISTENCIA SOCIAL                      |FEMENINO       |2  |0       |
|Acapulco de Juárez|COVID                                  |FEMENINO       |1  |0       |
|Acapulco de Juárez|INFORMACIÓN SOBRE TRÁMITES             |FEMENINO       |1  |0       |
|Acapulco de Juárez|TIERRA, PREDIOS Y USO DE SUELO         |FEMENINO       |1  |0       |
|Acapulco de Juárez|TOTAL_TS                               |FEMENINO       |8  |2       |
|Acapulco de Juárez|TRAMITES / INFO VEHICULAR              |FEMENINO       |3  |0       |
|Acapulco de Juárez|COVID                                  |MASCULINO      |1  |0       |
|Acapulco de Juárez|INFORMACIÓN SOBRE TRÁMITES             |MASCULINO      |1  |0       |
|Acapulco 

In [18]:
sqlstr = """
select ciudad, tema_solicitud, sexo, nr, grupo_id
from (
    select coalesce(s.d_ciudad, 'TOTAL_CD') as ciudad,
    coalesce(l.tema_solicitud, 'TOTAL_TS') as tema_solicitud,
    coalesce(l.sexo, 'TOTAL_SX') as sexo,
    count(*) as nr,
    grouping_id(s.d_ciudad, l.tema_solicitud, l.sexo) as grupo_id
    from locatel l
    join sepomex s on(l.d_codigo = s.d_codigo)
    group by cube(s.d_ciudad, l.tema_solicitud, l.sexo)
    ) x
order by 1, 3, grupo_id, 2
"""

spark.sql(sqlstr).show(80, truncate = False)

+------------------+---------------------------------------+---------------+---+--------+
|ciudad            |tema_solicitud                         |sexo           |nr |grupo_id|
+------------------+---------------------------------------+---------------+---+--------+
|Acapulco de Juárez|ASISTENCIA SOCIAL                      |FEMENINO       |2  |0       |
|Acapulco de Juárez|COVID                                  |FEMENINO       |1  |0       |
|Acapulco de Juárez|INFORMACIÓN SOBRE TRÁMITES             |FEMENINO       |1  |0       |
|Acapulco de Juárez|TIERRA, PREDIOS Y USO DE SUELO         |FEMENINO       |1  |0       |
|Acapulco de Juárez|TRAMITES / INFO VEHICULAR              |FEMENINO       |3  |0       |
|Acapulco de Juárez|TOTAL_TS                               |FEMENINO       |8  |2       |
|Acapulco de Juárez|COVID                                  |MASCULINO      |1  |0       |
|Acapulco de Juárez|INFORMACIÓN SOBRE TRÁMITES             |MASCULINO      |1  |0       |
|Acapulco 

In [25]:
sqlstr = """
select ciudad, tema_solicitud, nr
from(
    select coalesce(s.d_ciudad, 'TOTAL') as ciudad,
    coalesce(l.tema_solicitud, 'TOTAL') as tema_solicitud,
    count(*) as nr,
    grouping_id(s.d_ciudad, l.tema_solicitud) as grupo_id
    from locatel l
    join sepomex s on(l.d_codigo = s.d_codigo)
    group by cube(s.d_ciudad, l.tema_solicitud)
    ) x
order by ciudad, grupo_id, tema_solicitud
"""

spark.sql(sqlstr).show(80, truncate = False)

+-----------------------------+---------------------------------------+---+
|ciudad                       |tema_solicitud                         |nr |
+-----------------------------+---------------------------------------+---+
|Acapulco de Juárez           |ASISTENCIA SOCIAL                      |2  |
|Acapulco de Juárez           |COVID                                  |2  |
|Acapulco de Juárez           |INFORMACIÓN SOBRE TRÁMITES             |2  |
|Acapulco de Juárez           |QUEJAS Y DENUNCIAS                     |4  |
|Acapulco de Juárez           |REGISTRO CIVIL                         |1  |
|Acapulco de Juárez           |TIERRA, PREDIOS Y USO DE SUELO         |1  |
|Acapulco de Juárez           |TRAMITES / INFO VEHICULAR              |6  |
|Acapulco de Juárez           |VEHÍCULOS AUTOMOTORES Y TAXIS          |3  |
|Acapulco de Juárez           |TOTAL                                  |21 |
|Acatlán de Juárez            |SOLICITUD/SEGURO DE DESEMPLEO          |1  |
|Acatlán de 

In [26]:
sqlstr = """
select ciudad, tema_solicitud, sexo, nr, grupo_id
from (
    select coalesce(s.d_ciudad, 'TOTAL_CD') as ciudad,
    coalesce(l.tema_solicitud, 'TOTAL_TS') as tema_solicitud,
    coalesce(l.sexo, 'TOTAL_SX') as sexo,
    count(*) as nr,
    grouping_id(s.d_ciudad, l.tema_solicitud, l.sexo) as grupo_id
    from locatel l
    join sepomex s on(l.d_codigo = s.d_codigo)
    group by cube(s.d_ciudad, l.tema_solicitud, l.sexo)
    ) x
order by ciudad, grupo_id, tema_solicitud
"""

spark.sql(sqlstr).show(80, truncate = False)

+------------------+---------------------------------------+---------------+---+--------+
|ciudad            |tema_solicitud                         |sexo           |nr |grupo_id|
+------------------+---------------------------------------+---------------+---+--------+
|Acapulco de Juárez|ASISTENCIA SOCIAL                      |FEMENINO       |2  |0       |
|Acapulco de Juárez|COVID                                  |MASCULINO      |1  |0       |
|Acapulco de Juárez|COVID                                  |FEMENINO       |1  |0       |
|Acapulco de Juárez|INFORMACIÓN SOBRE TRÁMITES             |FEMENINO       |1  |0       |
|Acapulco de Juárez|INFORMACIÓN SOBRE TRÁMITES             |MASCULINO      |1  |0       |
|Acapulco de Juárez|QUEJAS Y DENUNCIAS                     |MASCULINO      |4  |0       |
|Acapulco de Juárez|REGISTRO CIVIL                         |MASCULINO      |1  |0       |
|Acapulco de Juárez|TIERRA, PREDIOS Y USO DE SUELO         |FEMENINO       |1  |0       |
|Acapulco 

In [29]:
sqlstr = """
select ciudad, tema_solicitud, nr
from (
    select coalesce(s.d_ciudad, 'TOTAL_CD') as ciudad,
    coalesce(l.tema_solicitud, 'TOTAL_TS') as tema_solicitud,
    count(*) as nr,
    grouping_id(s.d_ciudad, l.tema_solicitud) as grupo_id
    from locatel l
    join sepomex s on(l.d_codigo = s.d_codigo)
    where s.d_ciudad in('Acapulco de Juárez', 'Acayucan', 'Actopan')
    group by cube(s.d_ciudad, l.tema_solicitud)
    ) x
where not(ciudad = 'TOTAL_CD' and tema_solicitud != 'TOTAL_TS')
order by ciudad, grupo_id, tema_solicitud
"""

spark.sql(sqlstr).show(80, truncate = False)

+------------------+--------------------------------+---+
|ciudad            |tema_solicitud                  |nr |
+------------------+--------------------------------+---+
|Acapulco de Juárez|ASISTENCIA SOCIAL               |2  |
|Acapulco de Juárez|COVID                           |2  |
|Acapulco de Juárez|INFORMACIÓN SOBRE TRÁMITES      |2  |
|Acapulco de Juárez|QUEJAS Y DENUNCIAS              |4  |
|Acapulco de Juárez|REGISTRO CIVIL                  |1  |
|Acapulco de Juárez|TIERRA, PREDIOS Y USO DE SUELO  |1  |
|Acapulco de Juárez|TRAMITES / INFO VEHICULAR       |6  |
|Acapulco de Juárez|VEHÍCULOS AUTOMOTORES Y TAXIS   |3  |
|Acapulco de Juárez|TOTAL_TS                        |21 |
|Acayucan          |ASISTENCIA SOCIAL               |1  |
|Acayucan          |REGISTRO PÚBLICO DE LA PROPIEDAD|1  |
|Acayucan          |TRAMITES / INFO VEHICULAR       |1  |
|Acayucan          |TOTAL_TS                        |3  |
|Actopan           |TRAMITES / INFO VEHICULAR       |1  |
|Actopan      

In [32]:
sqlstr = """
select ciudad, sexo, tema_solicitud, nr
from (
    select coalesce(s.d_ciudad, 'TOTAL_CD') as ciudad,
    coalesce(l.tema_solicitud, 'TOTAL_TS') as tema_solicitud,
    coalesce(l.sexo, 'TOTAL_SX') as sexo,
    count(*) as nr,
    grouping_id(s.d_ciudad, l.tema_solicitud, l.sexo) as grupo_id
    from locatel l
    join sepomex s on(l.d_codigo = s.d_codigo)
    where s.d_ciudad in('Acapulco de Juárez', 'Acayucan', 'Actopan')
    group by cube(s.d_ciudad, l.tema_solicitud, l.sexo)
    ) x
where not(ciudad = 'TOTAL_CD' and tema_solicitud != 'TOTAL_TS')
and not (ciudad = 'TOTAL_CD' and sexo != 'TOTAL_SX')
order by ciudad, grupo_id, sexo, tema_solicitud
"""

spark.sql(sqlstr).show(80, truncate = False)

+------------------+---------------+--------------------------------+---+
|ciudad            |sexo           |tema_solicitud                  |nr |
+------------------+---------------+--------------------------------+---+
|Acapulco de Juárez|FEMENINO       |ASISTENCIA SOCIAL               |2  |
|Acapulco de Juárez|FEMENINO       |COVID                           |1  |
|Acapulco de Juárez|FEMENINO       |INFORMACIÓN SOBRE TRÁMITES      |1  |
|Acapulco de Juárez|FEMENINO       |TIERRA, PREDIOS Y USO DE SUELO  |1  |
|Acapulco de Juárez|FEMENINO       |TRAMITES / INFO VEHICULAR       |3  |
|Acapulco de Juárez|MASCULINO      |COVID                           |1  |
|Acapulco de Juárez|MASCULINO      |INFORMACIÓN SOBRE TRÁMITES      |1  |
|Acapulco de Juárez|MASCULINO      |QUEJAS Y DENUNCIAS              |4  |
|Acapulco de Juárez|MASCULINO      |REGISTRO CIVIL                  |1  |
|Acapulco de Juárez|MASCULINO      |TRAMITES / INFO VEHICULAR       |2  |
|Acapulco de Juárez|MASCULINO      |VE